In [135]:
import numpy as np
from numba import cuda
from tabulate import tabulate
from time import time
import matplotlib.pyplot as plt
from numba.cuda.random import create_xoroshiro128p_states, xoroshiro128p_uniform_float32
import pandas as pd

In [136]:
TPB = 16
pi_size = 16
BPG = int(pi_size/TPB)

Функция подсчета значения Пи на CPU

In [137]:
def CPU_calc(pi_size, N):
    res = np.zeros(pi_size)
    for i in range(pi_size):
        x = np.random.uniform(size=N)
        y = np.random.uniform(size=N)
        z = x ** 2 + y ** 2 <= 1
        res[i] = 4.0 * sum(z) / N
    return res

Ядро для GPU

In [138]:
@cuda.jit 
def pi_calcul(res, rng):
  i = cuda.blockIdx.x * cuda.blockDim.x + cuda.threadIdx.x
  h = 0
  # xoroshiro128p_uniform_float32 - Возвращает случайное значение от 0. до 1.
  if i < len(rng):
    x = xoroshiro128p_uniform_float32(rng, i)
    y = xoroshiro128p_uniform_float32(rng, i)
    if x ** 2 + y ** 2 <= 1.0:
      h += 1
  cuda.atomic.add(res, 0, h)

Функция, выполняющая вычисление значения Пи на GPU

In [139]:
def GPU_calc(N):

  # create_xoroshiro128p_states - для генерации случайного массива на CUDA
  rng_states = create_xoroshiro128p_states(N, seed=1)
  res = [0]

  dev_res = cuda.to_device(res)
  pi_calcul[N, TPB](dev_res, rng_states)
  tmp_ = dev_res.copy_to_host()
  pi_ = 4 * tmp_[0] / N
  return pi_

In [140]:
# 10 итераций подсчета числа Пи с увеличением значения N на 10000 после каждой итерации
iteration_count = np.arange(1,11,1)
N = 10000

df = {"Время на GPU":np.zeros(len(iteration_count), dtype=float), 
      "Время на CPU":np.zeros(len(iteration_count), dtype=float),
      "Пи на GPU":np.zeros(len(iteration_count), dtype=float),
      "Пи на CPU":np.zeros(len(iteration_count), dtype=float),
      "Ускорение":np.zeros(len(iteration_count), dtype=float),
      "Кол-во точек": np.zeros(len(iteration_count), dtype=int)}

df = pd.DataFrame(df, index=iteration_count)

for i in iteration_count:
  cpu_start = time()
  cpu_pi = CPU_calc(pi_size, N).mean()
  cpu_time = time() - cpu_start

  gpu_start = time()
  gpu_pi = GPU_calc(N)
  gpu_time = time() - gpu_start
    
  df.loc[i, "Время на GPU"] = gpu_time
  df.loc[i, "Время на CPU"] = cpu_time
  df.loc[i, "Пи на GPU"] = gpu_pi
  df.loc[i, "Пи на CPU"] = cpu_pi
  df["Ускорение"] = df["Время на CPU"] / df["Время на GPU"]
  df.loc[i, "Кол-во точек"] = N

  N += 10000

df

,Время на GPU,Время на CPU,Пи на GPU,Пи на CPU,Ускорение,Кол-во точек
1,0.138691,0.324021,3.155200,3.146550,2.336282,10000
2,0.006691,0.621952,3.157800,3.143088,92.947125,20000
3,0.010715,0.937665,3.150800,3.141025,87.511493,30000
4,0.012931,1.347336,3.147100,3.140525,104.193392,40000
5,0.014725,1.594939,3.143040,3.141050,108.317017,50000
6,0.017428,1.894585,3.147067,3.140792,108.711260,60000
7,0.019401,2.189835,3.147543,3.142557,112.874625,70000
8,0.022182,2.561910,3.147450,3.141862,115.497528,80000
9,0.026840,2.863565,3.145378,3.142606,106.688409,90000
10,0.027542,3.178224,3.143720,3.143162,115.397072,100000
